In [8]:
%load_ext Cython

In [1]:
import sys
from timeit import timeit
from tf.app import use
from pack import deepSize
from sparse import SparseString

In [81]:
%%cython

from array import array
from bisect import bisect_right as br, bisect_left as bl

NONE_INT = -1
NONE_STR = "\x00"
NONE_BYTE = b"\x00"
UTF8 = "utf8"

INTS = dict(
    B=("char", 1, False),
    b=("signed char", 1, True),
    H=("unsigned short", 2, False),
    h=("short", 2, True),
    I=("unsigned int", 4, False),
    i=("int", 4, True),
    Q=("unsigned int", 8, False),
    q=("int", 8, True),
)


def makeBoundaries():
    minpoints = []
    maxpoints = []
    signedmaxpoints = []
    typeindex = {}
    for (tp, (name, nbytes, signed)) in INTS.items():
        topvalue = 256 ** nbytes
        if signed:
            minvalue = topvalue // 2
            maxvalue = topvalue // 2 - 1
        else:
            minvalue = 0
            maxvalue = topvalue - 1
        if signed:
            typeindex[minvalue] = tp
            minpoints.append(minvalue)
            signedmaxpoints.append(maxvalue)
        else:
            maxpoints.append(maxvalue)
        typeindex[maxvalue] = tp
    minpoints = sorted(minpoints)
    maxpoints = sorted(maxpoints)
    signedmaxpoints = sorted(signedmaxpoints)
    typerank = {
        tp: i
        for (i, tp) in enumerate(
            sorted((t for t in INTS if INTS[t][2]), key=lambda t: INTS[t][1])
        )
    }

    def getType(minv, maxv):
        if minv >= 0:
            pos = bl(maxpoints, maxv)
            return None if pos >= len(maxpoints) else typeindex[maxpoints[pos]]
        posmin = bl(minpoints, -minv)
        if posmin >= len(minpoints):
            return None
        if maxv <= 0:
            return typeindex[minpoints[posmin]]
        posmax = bl(signedmaxpoints, maxv)
        if posmax >= len(signedmaxpoints):
            return None
        typemin = typeindex[minpoints[posmin]]
        typemax = typeindex[signedmaxpoints[posmax]]
        return typemin if typerank[typemin] > typerank[typemax] else typemax

    return getType


getIntSpec = makeBoundaries()

class CSparseString:
    def __init__(self, items):
        """Create sparse data container for (index, string) pairs.

        The pairs must be sorted by index, and no duplicate indices may occur.
        The set of indices must be a set of numbers, not necessarily consecutive.
        The values must be all strings (Unicode UTF8).

        Only indices where values change are stored.
        """

        if not items:
            self.indices = []
            return

        cdef list indices = []
        cdef list offsets = []
        cdef list bounds = []
        cdef list values = [NONE_BYTE]
        cdef dict valueMap = {NONE_STR: (0, 1)}
        cdef int curPos = 1
        cdef str prevV = NONE_STR
        cdef int prevI = -1

        def addValue(str v):
            nonlocal curPos

            if v in valueMap:
                (offset, bound) = valueMap[v]
                offsets.append(offset)
                bounds.append(bound)
            else:
                offsets.append(curPos)
                vb = v.encode(UTF8)
                bound = curPos + len(vb)
                bounds.append(bound)
                values.append(vb)
                valueMap[v] = (curPos, bound)
                curPos += len(vb)

        cdef int i
        cdef str v
        for (i, v) in items:
            if prevI == -1:
                indices.append(i)
                addValue(v)
            elif i != prevI + 1:
                indices.append(prevI + 1)
                addValue(NONE_STR)
                if v is not None:
                    indices.append(i)
                    addValue(v)
            elif prevV != v:
                indices.append(i)
                addValue(v)
            prevV = v
            prevI = i
        if prevV != NONE_STR:
            indices.append(prevI + 1)
            addValue(NONE_STR)

        offsets.append(curPos)
        self.values = b"".join(values)

        iTp = getIntSpec(min(indices), max(indices))
        self.indices = array(iTp, indices)
        self.nIndices = len(indices)
        oTp = getIntSpec(min(offsets), max(offsets))
        self.offsets = array(oTp, offsets)
        bTp = getIntSpec(min(bounds), max(bounds))
        self.bounds = array(bTp, bounds)

    def size(self):
        indices = self.indices
        offsets = self.offsets
        bounds = self.bounds
        values = self.values

        iSize = indices.itemsize * len(indices)
        oSize = offsets.itemsize * len(offsets)
        bSize = bounds.itemsize * len(bounds)
        vSize = len(values)
        return f"""{iSize + oSize + + bSize + vSize:>8}
\t{'indices':<8}: {indices.typecode}: {iSize:>8}
\t{'offsets':<8}: {offsets.typecode}: {oSize:>8}
\t{'bounds':<8}: {bounds.typecode}: {bSize:>8}
\t{'values':<8}: x: {vSize:>8}
"""

    def get(self, int index):
        cdef int nIndices = self.nIndices

        cdef int pos = br(self.indices, index)
        if pos == 0:
            return None
        pos = nIndices - 1 if pos > nIndices else pos - 1

        cdef int offset = self.offsets[pos]
        if offset == 0:
            return None
        cdef int bound = self.bounds[pos]
        rawValue = self.values[offset:bound]
        return rawValue.decode(UTF8)

In [85]:
%%cython

def test(provider):
    items = [(5, "foo"), (10, "bar")]
    dataC = provider(items)

    for i in range(12):
        print(i, dataC.get(i))

In [86]:
test(CSparseString)

0 None
1 None
2 None
3 None
4 None
5 foo
6 None
7 None
8 None
9 None
10 bar
11 None


In [50]:
A = use('bhsa:clone', silent='deep')

In [90]:
%%cython

import sys
from timeit import timeit
from tf.app import use
from pack import deepSize
from sparse import SparseString

def testPerformance(A, Cprovider, feat, check=True, withC=True):
    api = A.api
    maxNode = api.F.otype.maxNode
    Fs = api.Fs
    featObj = Fs(feat)
    items = sorted(featObj.items())
    firstI = items[0][0]
    halfI = items[len(items) // 2][0]
    lastI = items[-1][0]
    nonI = lastI + 100

    dataS = SparseString(items)
    if withC:
        dataCS = Cprovider(items)

    tfLookup = featObj.v
    spLookup = dataS.get
    if withC:
        cspLookup = dataCS.get

    print(f"{len(items)} items")
    print(f"Some items:")
    print(f"\tfirst: {firstI:>7} = {tfLookup(firstI)}")
    print(f"\thalf : {halfI:>7} = {tfLookup(halfI)}")
    print(f"\tlast : {lastI:>7} = {tfLookup(lastI)}")
    print(f"\tnon  : {nonI:>7} = {tfLookup(nonI)}")
    print(f"Size (TF compiled) = {deepSize(featObj.data):>8}")
    print(f"Size (Sparse)      = {dataS.size()}")
    if withC:
        print(f"Size (CSparse)     = {dataCS.size()}")

    if check:
        print("checking correctness ...")
        errors = []
        for i in range(maxNode + 5):
            tv = tfLookup(i)
            sv = spLookup(i)
            if withC:
                csv = cspLookup(i)
            if (withC and (tv != sv or tv != csv)) or ((not withC) and tv != sv):
                errors.append(i)
        if errors:
            print(f"{len(errors)} errors")
            for i in errors[0:10]:
                print(
                    f'''{i:>7} TF: "{tfLookup(i)}" SP: "{spLookup(i)}" SCP: "{cspLookup(i)}"'''
                    if withC
                    else f'''{i:>7} TF: "{tfLookup(i)}" SP: "{spLookup(i)}"'''
                )
        else:
            print("all values correct")
    else:
        print("correctness not checked")

    def execute(method, v):
        upperIndex = 430000
        key0 = 739  # not in the data of nametype
        key1 = 740  # in the data of nametype
        fI = firstI
        hI = halfI
        lI = lastI
        nI = nonI

        def doall():
            n = 0
            for i in range(fI - 10, lI + 10):
                if v(i) is not None:
                    n += 1

        times1 = 1000000
        times2 = max((1, 1000000 // (lI - fI)))

        print(method)
        for task in (
            ("first", "v(fI)", times1),
            ("half", "v(hI)", times1),
            ("last", "v(lI)", times1),
            ("non", "v(nI)", times1),
            ("all", "doall()", times2),
        ):
            (label, code, times) = task
            sys.stdout.write(f"\t{label:<5} {times:>7}x")
            sys.stdout.flush()
            t = timeit(code, globals=locals(), number=times)
            sys.stdout.write(f"  {t:>.4f}\n")

    execute("TF", tfLookup)
    execute("SP", spLookup)
    if withC:
        execute("CSP", cspLookup)

In [91]:
testPerformance(A, CSparseString, "nametype", withC=True)

38184 items
Some items:
	first:     740 = pers
	half :  210677 = topo
	last : 1446794 = pers
	non  : 1446894 = None
Size (TF compiled) =  2380461
Size (Sparse)      =   418826
	indices : I:   279176
	offsets : B:    69795
	bounds  : B:    69794
	values  : x:       61

Size (CSparse)     =   418826
	indices : I:   279176
	offsets : B:    69795
	bounds  : B:    69794
	values  : x:       61

checking correctness ...
all values correct
TF
	first 1000000x  0.1868
	half  1000000x  0.1948
	last  1000000x  0.2001
	non   1000000x  0.1536
	all         1x  0.2153
SP
	first 1000000x  1.1836
	half  1000000x  1.2711
	last  1000000x  1.2097
	non   1000000x  0.8810
	all         1x  1.3430
CSP
	first 1000000x  0.9839
	half  1000000x  1.0638
	last  1000000x  0.9570
	non   1000000x  0.7123
	all         1x  1.1039


In [107]:
%%cython

import sys
from array import array
from bisect import bisect_right as br, bisect_left as bl
from timeit import timeit
from pack import deepSize
from sparse import SparseString


NONE_INT = -1
NONE_STR = "\x00"
NONE_BYTE = b"\x00"
UTF8 = "utf8"

INTS = dict(
    B=("char", 1, False),
    b=("signed char", 1, True),
    H=("unsigned short", 2, False),
    h=("short", 2, True),
    I=("unsigned int", 4, False),
    i=("int", 4, True),
    Q=("unsigned int", 8, False),
    q=("int", 8, True),
)


def makeBoundaries():
    minpoints = []
    maxpoints = []
    signedmaxpoints = []
    typeindex = {}
    for (tp, (name, nbytes, signed)) in INTS.items():
        topvalue = 256 ** nbytes
        if signed:
            minvalue = topvalue // 2
            maxvalue = topvalue // 2 - 1
        else:
            minvalue = 0
            maxvalue = topvalue - 1
        if signed:
            typeindex[minvalue] = tp
            minpoints.append(minvalue)
            signedmaxpoints.append(maxvalue)
        else:
            maxpoints.append(maxvalue)
        typeindex[maxvalue] = tp
    minpoints = sorted(minpoints)
    maxpoints = sorted(maxpoints)
    signedmaxpoints = sorted(signedmaxpoints)
    typerank = {
        tp: i
        for (i, tp) in enumerate(
            sorted((t for t in INTS if INTS[t][2]), key=lambda t: INTS[t][1])
        )
    }

    def getType(minv, maxv):
        if minv >= 0:
            pos = bl(maxpoints, maxv)
            return None if pos >= len(maxpoints) else typeindex[maxpoints[pos]]
        posmin = bl(minpoints, -minv)
        if posmin >= len(minpoints):
            return None
        if maxv <= 0:
            return typeindex[minpoints[posmin]]
        posmax = bl(signedmaxpoints, maxv)
        if posmax >= len(signedmaxpoints):
            return None
        typemin = typeindex[minpoints[posmin]]
        typemax = typeindex[signedmaxpoints[posmax]]
        return typemin if typerank[typemin] > typerank[typemax] else typemax

    return getType


getIntSpec = makeBoundaries()


class CSparseString:
    def __init__(self, items):
        """Create sparse data container for (index, string) pairs.

        The pairs must be sorted by index, and no duplicate indices may occur.
        The set of indices must be a set of numbers, not necessarily consecutive.
        The values must be all strings (Unicode UTF8).

        Only indices where values change are stored.
        """

        if not items:
            self.indices = []
            return

        cdef list indices = []
        cdef list offsets = []
        cdef list bounds = []
        cdef list values = [NONE_BYTE]
        cdef dict valueMap = {NONE_STR: (0, 1)}
        curPos = 1
        cdef str prevV = NONE_STR
        cdef int prevI = -1

        def addValue(str v):
            nonlocal curPos

            if v in valueMap:
                (offset, bound) = valueMap[v]
                offsets.append(offset)
                bounds.append(bound)
            else:
                offsets.append(curPos)
                vb = v.encode(UTF8)
                bound = curPos + len(vb)
                bounds.append(bound)
                values.append(vb)
                valueMap[v] = (curPos, bound)
                curPos += len(vb)

        cdef int i
        cdef str v
        for (i, v) in items:
            if prevI == -1:
                indices.append(i)
                addValue(v)
            elif i != prevI + 1:
                indices.append(prevI + 1)
                addValue(NONE_STR)
                if v is not None:
                    indices.append(i)
                    addValue(v)
            elif prevV != v:
                indices.append(i)
                addValue(v)
            prevV = v
            prevI = i
        if prevV != NONE_STR:
            indices.append(prevI + 1)
            addValue(NONE_STR)

        offsets.append(curPos)
        
        self.values = b"".join(values)

        iTp = getIntSpec(min(indices), max(indices))
        self.indices = array(iTp, indices)
        self.nIndices = len(indices)
        oTp = getIntSpec(min(offsets), max(offsets))
        self.offsets = array(oTp, offsets)
        bTp = getIntSpec(min(bounds), max(bounds))
        self.bounds = array(bTp, bounds)

    def size(self):
        indices = self.indices
        offsets = self.offsets
        bounds = self.bounds
        values = self.values

        iSize = indices.itemsize * len(indices)
        oSize = offsets.itemsize * len(offsets)
        bSize = bounds.itemsize * len(bounds)
        vSize = len(values)
        return f"""{iSize + oSize + + bSize + vSize:>8}
\t{'indices':<8}: {indices.typecode}: {iSize:>8}
\t{'offsets':<8}: {offsets.typecode}: {oSize:>8}
\t{'bounds':<8}: {bounds.typecode}: {bSize:>8}
\t{'values':<8}: x: {vSize:>8}
"""

    def get(self, int index):
        cdef int nIndices = self.nIndices

        cdef int pos = br(self.indices, index)
        if pos == 0:
            return None
        pos = nIndices - 1 if pos > nIndices else pos - 1

        cdef int offset = self.offsets[pos]
        if offset == 0:
            return None
        cdef int bound = self.bounds[pos]
        cdef bytes rawValue = self.values[offset:bound]
        return rawValue.decode(UTF8)
    
    
def testPerformance(A, feat, check=True, withC=True):
    api = A.api
    maxNode = api.F.otype.maxNode
    Fs = api.Fs
    featObj = Fs(feat)
    cdef list items = sorted(featObj.items())
    cdef int firstI = items[0][0]
    cdef int halfI = items[len(items) // 2][0]
    cdef int lastI = items[-1][0]
    cdef int nonI = lastI + 100

    dataS = SparseString(items)
    if withC:
        dataCS = CSparseString(items)

    tfLookup = featObj.v
    spLookup = dataS.get
    if withC:
        cspLookup = dataCS.get

    print(f"{len(items)} items")
    print(f"Some items:")
    print(f"\tfirst: {firstI:>7} = {tfLookup(firstI)}")
    print(f"\thalf : {halfI:>7} = {tfLookup(halfI)}")
    print(f"\tlast : {lastI:>7} = {tfLookup(lastI)}")
    print(f"\tnon  : {nonI:>7} = {tfLookup(nonI)}")
    print(f"Size (TF compiled) = {deepSize(featObj.data):>8}")
    print(f"Size (Sparse)      = {dataS.size()}")
    if withC:
        print(f"Size (CSparse)     = {dataCS.size()}")

    cdef int i
    cdef str tv
    cdef str sv
    cdef str csv
    
    if check:
        print("checking correctness ...")
        errors = []
        for i in range(maxNode + 5):
            tv = tfLookup(i)
            sv = spLookup(i)
            if withC:
                csv = cspLookup(i)
            if (withC and (tv != sv or tv != csv)) or ((not withC) and tv != sv):
                errors.append(i)
        if errors:
            print(f"{len(errors)} errors")
            for i in errors[0:10]:
                print(
                    f'''{i:>7} TF: "{tfLookup(i)}" SP: "{spLookup(i)}" SCP: "{cspLookup(i)}"'''
                    if withC
                    else f'''{i:>7} TF: "{tfLookup(i)}" SP: "{spLookup(i)}"'''
                )
        else:
            print("all values correct")
    else:
        print("correctness not checked")

    def execute(method, v):
        cdef int upperIndex = 430000
        cdef int fI = firstI
        cdef int hI = halfI
        cdef int lI = lastI
        cdef int nI = nonI

        def doall():
            cdef int n = 0
            cdef int i
            for i in range(fI - 10, lI + 10):
                if v(i) is not None:
                    n += 1

        cdef int times1 = 1000000
        cdef int times2 = max((1, 1000000 // (lI - fI)))

        print(method)
        for task in (
            ("first", "v(fI)", times1),
            ("half", "v(hI)", times1),
            ("last", "v(lI)", times1),
            ("non", "v(nI)", times1),
            ("all", "doall()", times2),
        ):
            (label, code, times) = task
            sys.stdout.write(f"\t{label:<5} {times:>7}x")
            sys.stdout.flush()
            t = timeit(code, globals=locals(), number=times)
            sys.stdout.write(f"  {t:>.4f}\n")

    execute("TF", tfLookup)
    execute("SP", spLookup)
    if withC:
        execute("CSP", cspLookup)

In [108]:
testPerformance(A, "nametype", withC=True)

38184 items
Some items:
	first:     740 = pers
	half :  210677 = topo
	last : 1446794 = pers
	non  : 1446894 = None
Size (TF compiled) =  2380461
Size (Sparse)      =   418826
	indices : I:   279176
	offsets : B:    69795
	bounds  : B:    69794
	values  : x:       61

Size (CSparse)     =   418826
	indices : I:   279176
	offsets : B:    69795
	bounds  : B:    69794
	values  : x:       61

checking correctness ...
all values correct
TF
	first 1000000x  0.2141
	half  1000000x  0.2181
	last  1000000x  0.2258
	non   1000000x  0.1461
	all         1x  0.2066
SP
	first 1000000x  1.1950
	half  1000000x  1.2258
	last  1000000x  1.1760
	non   1000000x  0.8767
	all         1x  1.3299
CSP
	first 1000000x  0.9544
	half  1000000x  0.9778
	last  1000000x  0.9391
	non   1000000x  0.7301
	all         1x  1.0813


In [117]:
%%cython

cdef class Test:
    cdef int offset
    
    def __cinit__(self, int x):
        self.offset = x
        
    cpdef public int translate(self, int y):
        return self.offset + y

In [118]:
t = Test(5)

In [119]:
t.translate(10)

15

In [177]:
%%cython

import sys
from array import array
from bisect import bisect_left as bl


cdef int NONE_INT = -1
cdef str NONE_STR = "\x00"
cdef bytes NONE_BYTE = b"\x00"
cdef str UTF8 = "utf8"

INTS = dict(
    B=("char", 1, False),
    b=("signed char", 1, True),
    H=("unsigned short", 2, False),
    h=("short", 2, True),
    I=("unsigned int", 4, False),
    i=("int", 4, True),
    Q=("unsigned int", 8, False),
    q=("int", 8, True),
)


def makeBoundaries():
    minpoints = []
    maxpoints = []
    signedmaxpoints = []
    typeindex = {}
    for (tp, (name, nbytes, signed)) in INTS.items():
        topvalue = 256 ** nbytes
        if signed:
            minvalue = topvalue // 2
            maxvalue = topvalue // 2 - 1
        else:
            minvalue = 0
            maxvalue = topvalue - 1
        if signed:
            typeindex[minvalue] = tp
            minpoints.append(minvalue)
            signedmaxpoints.append(maxvalue)
        else:
            maxpoints.append(maxvalue)
        typeindex[maxvalue] = tp
    minpoints = sorted(minpoints)
    maxpoints = sorted(maxpoints)
    signedmaxpoints = sorted(signedmaxpoints)
    typerank = {
        tp: i
        for (i, tp) in enumerate(
            sorted((t for t in INTS if INTS[t][2]), key=lambda t: INTS[t][1])
        )
    }

    def getType(minv, maxv):
        if minv >= 0:
            pos = bl(maxpoints, maxv)
            return "" if pos >= len(maxpoints) else typeindex[maxpoints[pos]]
        posmin = bl(minpoints, -minv)
        if posmin >= len(minpoints):
            return ""
        if maxv <= 0:
            return typeindex[minpoints[posmin]]
        posmax = bl(signedmaxpoints, maxv)
        if posmax >= len(signedmaxpoints):
            return ""
        typemin = typeindex[minpoints[posmin]]
        typemax = typeindex[signedmaxpoints[posmax]]
        return typemin if typerank[typemin] > typerank[typemax] else typemax

    return getType


getIntSpec = makeBoundaries()


def makeData(items):
    if not items:
        return ((), (), (), b"")
    
    cdef list indices = []
    cdef list offsets = []
    cdef list bounds = []
    cdef list values = [NONE_BYTE]
    cdef dict valueMap = {NONE_STR: (0, 1)}
    cdef int curPos = 1
    cdef str prevV = NONE_STR
    cdef int prevI = -1

    def addValue(str v):
        nonlocal curPos

        if v in valueMap:
            (offset, bound) = valueMap[v]
            offsets.append(offset)
            bounds.append(bound)
        else:
            offsets.append(curPos)
            vb = v.encode(UTF8)
            bound = curPos + len(vb)
            bounds.append(bound)
            values.append(vb)
            valueMap[v] = (curPos, bound)
            curPos += len(vb)

    cdef int i
    cdef str v

    for (i, v) in items:
        if prevI == -1:
            indices.append(i)
            addValue(v)
        elif i != prevI + 1:
            indices.append(prevI + 1)
            addValue(NONE_STR)
            if v is not None:
                indices.append(i)
                addValue(v)
        elif prevV != v:
            indices.append(i)
            addValue(v)
        prevV = v
        prevI = i
    if prevV != NONE_STR:
        indices.append(prevI + 1)
        addValue(NONE_STR)

    offsets.append(curPos)

    valuesBin = b"".join(values)
    cdef str iTp = getIntSpec(min(indices), max(indices))
    indicesBin = array(iTp, indices)
    cdef str oTp = getIntSpec(min(offsets), max(offsets))
    offsetsBin = array(oTp, offsets)
    cdef str bTp = getIntSpec(min(bounds), max(bounds))
    boundsBin = array(bTp, bounds)
    return (indicesBin, offsetsBin, boundsBin, valuesBin)

In [178]:
getIntSpec(740, 1446795)

'I'

In [179]:
data = makeData(sorted(A.api.F.nametype.items()))

In [184]:
%%cython

from array import array
from bisect import bisect_right as br

cdef str UTF8 = "utf8"

cdef class CSparseString:
    cdef indices
    cdef nIndices
    cdef offsets
    cdef bounds
    cdef values
    
    def __cinit__(self, indices, offsets, bounds, values):
        self.indices = indices
        self.nIndices = len(indices)
        self.offsets = offsets
        self.bounds = bounds
        self.values = values
        
    cpdef public str size(self):
        indices = self.indices
        offsets = self.offsets
        bounds = self.bounds
        values = self.values

        iSize = indices.itemsize * len(indices)
        oSize = offsets.itemsize * len(offsets)
        bSize = bounds.itemsize * len(bounds)
        vSize = len(values)
        return f"""{iSize + oSize + + bSize + vSize:>8}
\t{'indices':<8}: {indices.typecode}: {iSize:>8}
\t{'offsets':<8}: {offsets.typecode}: {oSize:>8}
\t{'bounds':<8}: {bounds.typecode}: {bSize:>8}
\t{'values':<8}: x: {vSize:>8}
"""

    cpdef public str get(self, int index):
        cdef int nIndices = self.nIndices

        cdef int pos = br(self.indices, index)
        if pos == 0:
            return None
        pos = nIndices - 1 if pos > nIndices else pos - 1

        cdef int offset = self.offsets[pos]
        if offset == 0:
            return None
        cdef int bound = self.bounds[pos]
        cdef bytes rawValue = self.values[offset:bound]
        return rawValue.decode(UTF8)

In [185]:
CSP = CSparseString(*data)

In [186]:
for i in range(730, 750):
    print(i, CSP.get(i))

730 None
731 None
732 None
733 None
734 None
735 None
736 None
737 None
738 None
739 None
740 pers
741 None
742 None
743 None
744 None
745 None
746 None
747 None
748 None
749 None
